# TensorRider Algorithm

In [1]:
import numpy as np
import tensorflow as tf
import tensorlayer as tl

In [2]:
sess = tf.InteractiveSession()

In [3]:
def rgb2mono(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    # mono = 0.2989 * r + 0.5870 * g + 0.1140 * b
    mono = 0.5 * r + 0.25 * g + 0.25 * b
    return mono

In [4]:
def prepareDataArrays(iterator):
    X = np.zeros(4800)
    y = np.zeros(1)
    recordCounter = 0;

    for string_record in iterator:
        recordCounter += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        imageString = (example.features.feature['image'].bytes_list.value[0])
        label = (example.features.feature['label'].int64_list.value[0])

        image = np.fromstring(imageString, dtype=np.uint8)
        image = image.reshape((60, 80, 3))
        image = rgb2mono(image)
        image = image.reshape((4800))

        X = np.vstack((X,image))
        y = np.append(y,label)
        if recordCounter % 100 == 0:
            print(recordCounter,end = '->')

    y = y.reshape((recordCounter + 1,))
#     y = np.round(y / 7) #Downsampling
#     y = y + 6
    y = np.round(y / 12)
    y = y + 3
    return X, y

In [5]:
trainIterator = tf.python_io.tf_record_iterator(path="train.tfrecords")
valIterator = tf.python_io.tf_record_iterator(path="val.tfrecords")
testIterator = tf.python_io.tf_record_iterator(path="test.tfrecords")

# TFRecords -> Arrays

In [ ]:
print("\nTrain...")
X_train, y_train = prepareDataArrays(trainIterator)
print("\nVal...")
X_val, y_val = prepareDataArrays(valIterator)
print("\nTest...")
X_test, y_test = prepareDataArrays(testIterator)


Train...
100->

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


200->300->400->500->600->

# Arrays -> NPZ File

In [7]:
np.savez("arrays.npz", xtr = X_train, ytr = y_train, xval = X_val, yval = y_val, xt = X_test, yt = y_test)

# NPZ File -> Arrays

In [6]:
npRecall = np.load("arrays.npz")
X_train = npRecall["xtr"]
y_train = npRecall["ytr"]
X_val = npRecall["xval"]
y_val = npRecall["yval"]
X_test = npRecall["xt"]
y_test = npRecall["yt"]

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 4800], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

# Define the Neural Network

In [8]:
network = tl.layers.InputLayer(x, name='input')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 2048, tf.nn.sigmoid, name='sigmoid1')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 1024, tf.nn.sigmoid, name='sigmoid2')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, 1024, tf.nn.sigmoid, name='sigmoid3')
network = tl.layers.DenseLayer(network, 512, tf.nn.sigmoid, name='sigmoid4')
# network = tl.layers.DenseLayer(network, n_units=13, act=tf.identity, name='output')
network = tl.layers.DenseLayer(network, n_units=7, act=tf.identity, name='output')

[TL] InputLayer  input: (?, 4800)
[TL] DropoutLayer drop1: keep:0.800000 is_fix:False
[TL] DenseLayer  sigmoid1: 2048 sigmoid
[TL] DenseLayer  sigmoid2: 1024 sigmoid
[TL] DenseLayer  sigmoid3: 1024 sigmoid
[TL] DenseLayer  sigmoid4: 512 sigmoid
[TL] DenseLayer  output: 7 identity


In [9]:
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

In [10]:
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

In [11]:
tl.layers.initialize_global_variables(sess)

In [12]:
network.print_params()

[TL]   param   0: sigmoid1/W:0         (4800, 2048)       float32_ref (mean: -7.808665395714343e-05, median: -7.939466740936041e-05, std: 0.08799178153276443)   
[TL]   param   1: sigmoid1/b:0         (2048,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   2: sigmoid2/W:0         (2048, 1024)       float32_ref (mean: -3.456316699157469e-05, median: -2.1145600840100087e-05, std: 0.08799448609352112)   
[TL]   param   3: sigmoid2/b:0         (1024,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   4: sigmoid3/W:0         (1024, 1024)       float32_ref (mean: 5.250635149423033e-05, median: 0.00011346317478455603, std: 0.08791771531105042)   
[TL]   param   5: sigmoid3/b:0         (1024,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   6: sigmoid4/W:0         (1024, 512)     

In [13]:
network.print_layers()

[TL]   layer   0: drop1/mul:0          (?, 4800)          float32
[TL]   layer   1: sigmoid1/Sigmoid:0   (?, 2048)          float32
[TL]   layer   2: sigmoid2/Sigmoid:0   (?, 1024)          float32
[TL]   layer   3: sigmoid3/Sigmoid:0   (?, 1024)          float32
[TL]   layer   4: sigmoid4/Sigmoid:0   (?, 512)           float32
[TL]   layer   5: output/Identity:0    (?, 7)             float32


# (Optional) Setting up TensorBoard

In [14]:
acc_summ = tf.summary.scalar('acc', acc)  
cost_summ = tf.summary.scalar('cost', cost)  
summary = tf.summary.merge_all()  
writer = tf.summary.FileWriter('./logs')  
writer.add_graph(sess.graph) 

# Start Training

In [15]:
tl.utils.fit(
    sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=100, n_epoch=500, print_freq=5, X_val=X_val, y_val=y_val, eval_train=False, tensorboard=True)

[TL] Setting up tensorboard ...
[TL] [!] logs/ exists ...
[TL] Param name sigmoid1/W:0


INFO:tensorflow:Summary name sigmoid1/W:0 is illegal; using sigmoid1/W_0 instead.


[TL] Param name sigmoid1/b:0


INFO:tensorflow:Summary name sigmoid1/b:0 is illegal; using sigmoid1/b_0 instead.


[TL] Param name sigmoid2/W:0


...

INFO:tensorflow:Summary name output/W:0 is illegal; using output/W_0 instead.


[TL] Param name output/b:0


INFO:tensorflow:Summary name output/b:0 is illegal; using output/b_0 instead.


[TL] Finished! use $tensorboard --logdir=logs/ to start server
[TL] Start training the network ...


KeyboardInterrupt: 

# Test the Neural Network

In [12]:
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

[TL] Start testing the network ...
[TL]    test loss: 0.669836
[TL]    test acc: 0.759868


# Save the Trained Model

In [13]:
tl.files.save_npz(network.all_params, name='model_old.npz')
sess.close()

[TL] [*] model.npz saved


# Load the Trained Model

In [14]:
tl.files.load_and_assign_npz(sess=sess, name='model_old.npz', network=network)

[TL] [*] Load model_old.npz SUCCESS!


# Controlling TensorRider Using the Generated Model

In [15]:
import cv2
import urllib.request
import numpy as np
import socket
import threading
from time import ctime,sleep
import string

remoteImage = np.array([])
stream = urllib.request.urlopen('http://192.168.73.73:8080/?action=stream&ignored.mjpg')
bytes = bytes()

In [16]:
def Image_Refreshing_Thread():
    global remoteImage
    global stream
    global bytes
    while True:
        bytes += stream.read(1024)
        a = bytes.find(b'\xff\xd8')
        b = bytes.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes[a:b+2]
            bytes = bytes[b+2:]
            i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            i = rgb2mono(i)
            remoteImage = i.reshape((1,4800))

In [17]:
def Controlling_Thread():
    global remoteImage
    addr=('192.168.73.73',51423)
    s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    while True:
        direction = tl.utils.predict(sess, network, remoteImage, x, y_op, batch_size=None)
        msgCtrl_Udp = str(50) + "," + str((direction[0] - 3) * 26)
#         print(msgCtrl_Udp)
        s.sendto(msgCtrl_Udp.encode('utf-8'), addr)
        sleep(0.1)

In [18]:
RefreshImageThread = threading.Thread(target = Image_Refreshing_Thread)
RefreshImageThread.start()

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [19]:
ControllingThread = threading.Thread(target = Controlling_Thread)
ControllingThread.start()

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':
